In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import keras
import PIL.Image
import IPython.display
import numpy as np


from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers
print(keras.__version__)
# K.tensorflow_backend.set_image_dim_ordering('th')
print(K.image_data_format())

# Deterministic output.
# Tired of seeing the same results every time? Remove the line below.
np.random.seed(1000)

# The results are a little better when the dimensionality of the random vector is only 10.
# The dimensionality has been left at 100 for consistency with other GAN implementations.
randomDim = 100

# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5


#Set to False to use DCGAN 
dense=False

# Optimizer
adam = Adam(lr=0.0002, beta_1=0.5)
if dense:
  X_train = X_train.reshape(60000, 784)
  generator = Sequential()
  generator.add(Dense(256, input_dim=randomDim))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense(512))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense(1024))
  generator.add(LeakyReLU(0.2))
  generator.add(Dense(784, activation='tanh'))

  discriminator = Sequential()
  discriminator.add(Dense(1024, input_dim=784))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(512))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(256))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Dense(1, activation='sigmoid'))

else:
  X_train = X_train.reshape(60000, 28,28,1)
  generator = Sequential()
  generator.add(Dense(128*7*7, input_dim=randomDim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  #generator.add(BatchNormalization(momentum=0.9)) 
  generator.add(LeakyReLU(0.2))
  generator.add(Reshape(( 7, 7,128)))
  generator.add(UpSampling2D(size=(2, 2)))
  generator.add(Conv2D(64, kernel_size=(5, 5), padding='same'))
  #generator.add(BatchNormalization(momentum=0.9)) 
  generator.add(LeakyReLU(0.2))
  generator.add(UpSampling2D(size=(2, 2)))
  generator.add(Conv2D(1, kernel_size=(5, 5), padding='same', activation='tanh'))
  generator.compile(loss='binary_crossentropy', optimizer=adam)

  discriminator = Sequential()
  discriminator.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', input_shape=(28, 28,1), kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same'))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  discriminator.add(Flatten())
  discriminator.add(Dense(1, activation='sigmoid'))

generator.summary()





discriminator.compile(loss='binary_crossentropy', optimizer=adam,metrics=['accuracy'])

discriminator.summary()

# We combined the generator and discriminator to create a gan model
discriminator.trainable = False
ganInput = Input(shape=(randomDim,))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=adam,metrics=['accuracy'])

dLosses = []
gLosses = []

# Plot the loss from each batch
def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('gan_loss_epoch_%d.png' % epoch)

# Create a wall of generated MNIST images
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)
    IPython.display.display(IPython.display.Image(data=('gan_generated_image_epoch_%d.png' % epoch)))
# Save the generator and discriminator networks (and weights) for later use


def saveModels(epoch):
    generator.save('gan_generator_epoch_%d.h5' % epoch)
    discriminator.save('gan_discriminator_epoch_%d.h5' % epoch)

epochs=22
batchSize=128
batchCount = X_train.shape[0] // batchSize
print('Epochs:', epochs)
print('Batch size:', batchSize)
print('Batches per epoch:', batchCount)

for e in range(1, epochs+1):
    print('-'*15, 'Epoch %d' % e, '-'*15)
    for _ in tqdm(range(batchCount)):
        ####### Train discriminator #####
        

        # Get a random set of input noise and images
        # Generate fake MNIST images
        noise = np.random.normal(0, 1, size=[batchSize, randomDim])
        
        generatedImages = generator.predict(noise)
        
        #we create a batch composed of real and fake images
        imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]
        
        X = np.concatenate([imageBatch, generatedImages])

        
        # We manually create the labels corresponding to X (1 for real, 0 for generated)
        yDis = np.zeros(2*batchSize)
        yDis[:batchSize] = 1.0

        # Train discriminator
        discriminator.trainable = True
        dloss,acc = discriminator.train_on_batch(X, yDis)
        

        ####### Train generator #####
        # Generate fake MNIST images
        noise = np.random.normal(0, 1, size=[batchSize, randomDim])
        yGen = np.ones(batchSize)
        # Train generator

        discriminator.trainable = False
        gloss,acc = gan.train_on_batch(noise, yGen)
        # hist = gan.train_on_batch(noise, yGen)
        
    # Store loss of most recent batch from this epoch
    dLosses.append(dloss)
    #gLosses.append(gloss)

    if e == 1 or e % 5 == 0:
        plotGeneratedImages(e)
        saveModels(e)

    # Plot losses from every epoch
    plotLoss(e)




2.4.3
channels_last
11493376/11490434 [==============================] - 0s 0us/step


  0%|          | 0/468 [00:00<?, ?it/s]

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6272)              633472    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 6272)              0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 64)        204864    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 28, 28, 64)        0

 83%|████████▎ | 389/468 [15:09<03:04,  2.34s/it]